In [26]:
import pandas as pd

## *Неважливо, наскільки добра буде модель - якщо дати їй неякісні дані, вона не буде працювати*

In [27]:
dataframe = pd.read_excel('../readyDatasets/2018-2023.xlsx')

cols = ['spec_num', 'Спеціальність', 'specialization']
dataframe['spec_full'] = dataframe[cols].fillna('').apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

dataframe.drop(columns=['spec_num', 'Спеціальність', 'specialization', 'Назва закладу', 'Фіксований обсяг',
                        'на загальних підставах', 'Усього рекомендовано'], inplace=True)

dataframe.dropna(inplace=True)

dataframe.columns = dataframe.columns.str.replace('\n', ' ')
dataframe['form'] = dataframe['form'].map({'Заочна': 0, 'Денна': 1})

У нашому датасеті є декілька категоріальних змінних: назва спеціальності, органу управління, номер університету та рік. Можна було б застосувати до них метод *pandas.get_dummies()*, щоб перетворити категоріальні змінні на фіктивні змінні, які являють собою числові змінні, що використовуються для представлення категоріальних даних. Але в нашому випадку такий підхід би значно збільшив розмір датасету (аж до 416 стовпчиків), що не є дуже добре. Тому будемо застовувати так званий Target encoding, який не вимагає створення додаткових стовпчиків.

# Target encoding
Target encoding передбачає заміну категоріальної ознаки середнім цільовим значенням усіх точок даних, що належать до категорії. 

Однією з проблем цільового кодування є перенавчання. Деякі також називають це витоком цільової змінної в одну з фіч (Leakage of target). У цих випадках модель із цільовим кодуванням погано узагальнює нові дані. Зменшити перенавчання при цільовому кодуванні можна за допомогою згладжування.

Одним із популярних методів згладжування є використання комбінації таргету для категорії та глобального цільового середнього для кожної точки даних. Ця техніка особливо корисна для вирішення ситуацій, коли для деяких категорій дуже мало даних.

## Аддитивне згладжування
$$
\mu=\frac{n \times \bar{x}+m \times w}{n+m}
$$
де
- $\mu$ --- середнє, яке ми намагаємося обчислити (те, яке замінить наші категоріальні значення)
- $n$ --- кількість елементів у групі
- $\bar{x}$ --- передбачуване середнє
- $m$ ---ваговий коефіцієнт, який застосовується для загального середньго значення
- $w$ --- загальне середнє значення

У цій формулі $m$ є єдиним параметром, який потрібно встановити. Ідея полягає в тому, що чим вищий $m$, тим більше ми покладаємося на загальне середнє $w$. Якщо $m$ дорівнює 0, тоді отримаємо емпіричне середнє, яке дорівнює:
$$
\mu=\frac{n \times \bar{x}+0 \times w}{n+0}=\frac{n \times \bar{x}}{n}=\bar{x}
$$
Іншими словами, в такому випадку згладжування не відбувається.

Sources: стаття [Target encoding done the right way](https://maxhalford.github.io/blog/target-encoding/) та [відео пояснення](https://www.youtube.com/watch?v=Bao9GGZMLhU).

In [28]:
def calc_smooth_mean(df, by, m=5):
    # Глобальне середнє
    mean = df['УСЬОГО'].mean()

    # Обчислюємо кількість значень і середнє для кожної групи
    agg = df.groupby(by)['УСЬОГО'].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Згладжені середні
    smooth = (counts * means + m * mean) / (counts + m)

    # Збережемо мапу на майбутнє
    df_map = pd.DataFrame()
    df_map[by] = df[by]
    df_map['smooth_mean'] = df[by].map(smooth)
    df_map.drop_duplicates(inplace=True)
    df_map.to_csv('../readyDatasets/map_'+by+'_to_smoothed_means.csv', index=False, sep='$')

    # Замінюємо кожне значення відповідним згладженим середнім
    return df[by].map(smooth)

In [29]:
for col in ['spec_full', 'Орган управління', 'uni_code', 'Рік']:
    dataframe[col] = calc_smooth_mean(dataframe, by=col)

In [36]:
dataframe.head()

,uni_code,form,Орган управління,Усього подано заяв,Подано заяв на бюджет,Допущено до конкурсу,Середній пріоритет допущених,Середній пріоритет рекомендованих,Суперобсяг,Рекомендовано за співбесідою,...,Сер. Бал (на загальних підставах),Макс. Бал (на загальних підставах),Рік,Макс. обсяг держзамовлення,СЕР,МІН,МАКС,Ліцензійний обсяг,Регіональний коефіцієнт,spec_full
0,10.255181,1,19.390828,373,315,310,2.954839,2.761905,525,0,...,164.857200,174.267,17.696067,21.0,144.8,113.4,190.9,50.0,1.04,15.667939
4,6.818932,1,19.390828,222,194,192,2.901042,2.000000,525,0,...,169.566000,182.564,17.696067,13.0,145.1,117.3,185.8,30.0,1.07,15.667939
5,9.035306,1,19.390828,499,435,429,3.095571,2.628571,525,0,...,170.650719,185.399,17.696067,35.0,149.5,110.8,194.2,95.0,1.00,15.667939
6,9.752988,1,19.390828,369,340,336,3.011905,1.840000,525,0,...,176.351348,194.243,17.696067,25.0,152.0,111.2,197.0,50.0,1.00,15.667939
7,6.771794,1,19.390828,208,177,173,2.716763,2.257143,525,0,...,159.607794,172.380,17.696067,35.0,144.9,111.9,192.2,50.0,1.00,15.667939


In [11]:
dataframe.to_csv('readyDatasets/preprocessed_dataframe.csv', index=False)